In [93]:
rm(list = ls())
library(MASS)
library(ncpen)
library(datamicroarray)
library(gsubfn)
set.seed(20200413)
par(mfrow = c(4, 4))
# describe_data()
# read samples

In [94]:
data("burczynski")
save.name = "burczynski"
xx.mat = as.matrix(burczynski$x)
xx.mat = sweep(xx.mat, 2, sqrt(colSums(xx.mat ^ 2) / nrow(xx.mat)), "/")
#colSums(xx.mat^2)
c.vec = ifelse(burczynski$y == levels(burczynski$y)[1], 1, 2)

In [95]:
p = dim(xx.mat)[2]
top.vec = 2^(2:4)*1e2
# top.vec = p

In [96]:
# SURE independence screening
n1 = sum(c.vec == 1)
n2 = sum(c.vec == 2)
n = n1 + n2
y.vec = ifelse(c.vec == 1, -n / n1, n / n2)
s.vec = NULL
for (j in 1:ncol(xx.mat)) {
  s.vec = c(s.vec, coef(glm(
    y.vec ~ ., data = data.frame(y.vec, xx.mat[, j]), family = "gaussian"
  ))[-1])
}
o.vec = order(abs(s.vec), decreasing = T)
coef(glm(y.vec ~ ., data = data.frame(y.vec, xx.mat[, o.vec[1]]), family =
           "gaussian"))[-1]
coef(glm(y.vec ~ ., data = data.frame(y.vec, xx.mat[, o.vec[10]]), family =
           "gaussian"))[-1]

xx.mat...o.vec.1.. 
         -6.089913

xx.mat...o.vec.10.. 
          -4.911896

In [99]:
# tuning by cv and then calculate leave-one-out errors
n.fold = 1e1
gam.size = c(1/3:1, 2:3)

In [101]:
viz <- F
err.list = ms.list = list()
for (top in top.vec) {
  x.mat = xx.mat[, o.vec[1:top]]
  err.mat = ms.mat = NULL
  for (sid in 1:nrow(x.mat)) {
    print(sid)
    # training/test samples
    tx.mat = x.mat[-sid, ]
    tc.vec = c.vec[-sid]
    n1 = sum(tc.vec == 1)
    n2 = sum(tc.vec == 2)
    n = n1 + n2
    nr = n2 / n1
    ty.vec = ifelse(tc.vec == 1, -n / n1, n / n2)
    nx.mat = x.mat[sid, , drop = F]
    ny.vec = ifelse(c.vec[sid] == 1, -n / n1, n / n2)
    m1.vec = colMeans(tx.mat[tc.vec == 1, ])
    m2.vec = colMeans(tx.mat[tc.vec == 2, ])
    s1.mat = var(tx.mat[tc.vec == 1, ])
    s2.mat = var(tx.mat[tc.vec == 2, ])
    ps.mat = ((n1 - 1) * s1.mat + (n2 - 1) * s2.mat) / (n - 2)
    # fold id for cross-validation
    fold.id = rep(0, n)
    suppressWarnings({fold.list = split(c(sample((1:n)[tc.vec == 1]), sample((1:n)[tc.vec == 2])), 1:n.fold)})
    for (i in 1:n.fold) {
      fold.id[fold.list[[i]]] = i
    }
    lam.max = max(abs(-t(tx.mat) %*% (ty.vec - mean(ty.vec)) / n))
    lam.vec = exp(seq(log(lam.max), log(lam.max / 100), length.out = 300))
    message(fn$identity('Fitting lasso...'))
    suppressWarnings({fit = cv.ncpen(
      ty.vec,
      tx.mat,
      family = "gaussian",
      penalty = "lasso",
      x.standardize = F,
      intercept = T,
      df.max = n,
      fold.id = fold.id,
      n.fold = n.fold,
      lambda = lam.vec
    )})
    if(viz) plot(fit$rmse)
    b.mat = coef(fit)$beta[-1]
    df.max = sum(b.mat != 0)
    # clipped lasso
    opt.gam = coef(fit)$lambda
    lam.vec = fit$lambda
    
    g.vec = opt.gam * gam.size
    tb.mat = NULL
    for (gam in g.vec) {
      if (max(lam.vec) > gam) {
        clam.vec = exp(seq(log(max(lam.vec)), log(gam), length.out = 100))
        clam.vec[100] = gam
      } else {
        clam.vec = exp(seq(log(2 * gam), log(gam), length.out = 10))
      }
      fit = cv.ncpen(
        ty.vec,
        tx.mat,
        family = "gaussian",
        penalty = "classo",
        lambda = clam.vec,
        gamma = gam,
        x.standardize = F,
        intercept = T,
        df.max = df.max + 30,
        tau = 1.1,
        fold.id = fold.id,
        n.fold = n.fold
      )
      if(viz) plot(fit$rmse)
      b.mat = cbind(b.mat, coef(fit)$beta[-1])
    }
    # mcp
    t.vec = 1:3 + 0.1
    tb.mat = NULL
    for (tau in t.vec) {
      fit = cv.ncpen(
        ty.vec,
        tx.mat,
        family = "gaussian",
        penalty = "mcp",
        x.standardize = F,
        intercept = T,
        df.max = df.max + 30,
        tau = tau,
        fold.id = fold.id,
        n.fold = n.fold
      )
      if(viz) plot(fit$rmse)
      b.mat = cbind(b.mat, coef(fit)$beta[-1])
    }
    # summary
    e.vec = NULL
    for (bid in 1:ncol(b.mat)) {
      tb.vec = b.mat[, bid]
      sgn = drop(t(m2.vec - m1.vec) %*% tb.vec)
      e.vec = c(e.vec, mean((ny.vec > 0) != (
        sign(sgn) * (
          nx.mat %*% tb.vec - drop(
            t(m2.vec + m1.vec) %*% tb.vec / 2 + t(tb.vec) %*% ps.mat %*% tb.vec / sgn *
              log(nr)
          )
        ) > 0
      )))
    }
    err.mat = rbind(err.mat, e.vec)
    ms.mat = rbind(ms.mat, colSums(b.mat != 0))
    print(c(top, colMeans(err.mat, na.rm = T)))
    print(c(top, colMeans(ms.mat, na.rm = T)))
  }
  err.list[[top]] = err.mat
  ms.list[[top]] = ms.mat
}
save.image(fn$identity('`save.name`2.SURE.`n.fold`f.loo.RData'))

[1] 1


Fitting lasso...



 [1] 400   0   0   0   0   0   0   0   0   0
      b.mat                                                 
  400    59    94    83    58    34    27    18    12    17 
[1] 2


Fitting lasso...



 [1] 400   0   0   0   0   0   0   0   0   0
      b.mat                                                 
400.0  60.0  89.0  84.0  59.0  37.5  23.5  16.0  13.0  15.5 
[1] 3


...

Fitting lasso...



 [1] 400.00000000   0.04938272   0.06172840   0.04938272   0.06172840
 [6]   0.06172840   0.06172840   0.12345679   0.09876543   0.09876543
              b.mat                                                             
400.00000  61.82716  82.01235  75.07407  58.20988  36.08642  25.70370  16.76543 
                    
 16.01235  16.16049 
[1] 82


Fitting lasso...



 [1] 400.00000000   0.04878049   0.06097561   0.04878049   0.06097561
 [6]   0.06097561   0.06097561   0.12195122   0.09756098   0.09756098
              b.mat                                                             
400.00000  61.79268  82.06098  74.98780  58.03659  36.02439  25.62195  16.80488 
                    
 16.06098  16.14634 
[1] 83


Fitting lasso...



In [89]:
load(fn$identity('`save.name`2.SURE.`n.fold`f.loo.RData'))

In [90]:
cn <- c('lasso', paste('cl.', round(gam.size,2), '*g', sep = ''), paste('mcp.t.', 1:3 + .1, sep = ''))

In [91]:
err.agg = round(rbind(colSums(err.list[[4e2]]), colSums(err.list[[8e2]]), colSums(err.list[[16e2]])), digits = 3)
ms.agg = round(rbind(colMeans(ms.list[[4e2]]), colMeans(ms.list[[8e2]]), colMeans(ms.list[[16e2]])), digits = 3)
colnames(ms.agg)<-colnames(err.agg) <- cn
agg <- rbind(cbind(m='err',top=top.vec,err.agg), cbind(m='ms',top=top.vec,ms.agg))
agg

m,top,lasso,cl.0.01*g,cl.0.12*g,cl.0.23*g,cl.0.34*g,cl.0.45*g,cl.0.56*g,cl.0.67*g,cl.0.78*g,cl.0.89*g,cl.1*g,mcp.t.1.1,mcp.t.2.1,mcp.t.3.1
err,400,7,16,12,10,9,10,9,8,9,9,7,16,12,13
err,800,7,9,15,10,8,7,6,7,6,6,9,12,11,14
err,1600,3,17,4,3,3,3,3,3,5,3,3,13,11,13
ms,400,60.598,16.102,58.882,77.528,80.638,77.063,72.307,65.929,62.85,57.709,56.457,15.819,15.307,15.394
ms,800,79.213,18.898,83.756,80.638,78.378,72.118,74.504,70.441,69.772,68.417,63.362,17.583,16.906,16.929
ms,1600,89.528,35.055,116.417,112.252,105.276,100.772,95.559,91.433,88.008,85.583,83.055,22.331,21.913,20.433
